# Multiagent CSV Data Analysis with Code Interpreter

This notebook demonstrates a multiagent system that:

- Scans CSV files from a local folder called `data`.
- For each CSV file, uploads it and uses a Code Interpreter–based agent (via the Azure AI Projects service) to perform data analysis (e.g. generating summary statistics and visualizations).
- Uses a second agent (via Semantic Kernel orchestration) to format and present the analysis results in a structured Markdown output.

Make sure you have the required packages installed (e.g., `azure-ai-projects`, `azure-identity`, and `semantic_kernel`) and that your environment variables (such as `PROJECT_CONNECTION_STRING` and `MODEL_DEPLOYMENT_NAME`) are set appropriately.


In [5]:
import os
import glob
import asyncio
import nest_asyncio
from pathlib import Path
from dotenv import load_dotenv
from IPython.display import display, HTML   

load_dotenv()  # Load environment variables from .env file

# Allow nested asyncio loops (helpful in notebooks)
nest_asyncio.apply()

# Import required libraries for the Code Interpreter agent
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FilePurpose, MessageRole, CodeInterpreterTool
from azure.identity import DefaultAzureCredential

# Set your configuration via environment variables (or hardcode for testing)
PROJECT_CONNECTION_STRING = os.getenv('AZURE_AI_AGENT_PROJECT_CONNECTION_STRING')
MODEL_DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')  
# Initialize the Azure AI Projects client
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=PROJECT_CONNECTION_STRING
)

print("Configuration loaded and project client initialized.")


Configuration loaded and project client initialized.


In [6]:
import os
import glob
import asyncio
import nest_asyncio
from pathlib import Path

# Allow nested asyncio loops (helpful in notebooks)
nest_asyncio.apply()

# Azure AI Projects and Identity Imports (for Code Interpreter tool)


# Semantic Kernel agent imports (if needed for local tool integration)
# (These are used here to mimic multi-tool capabilities in a single agent.)
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings

# Configuration: Ensure these environment variables are set in your environment.
PROJECT_CONNECTION_STRING = os.getenv("AZURE_AI_AGENT_PROJECT_CONNECTION_STRING")
MODEL_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

print("Configuration loaded.")


Configuration loaded.


In [2]:
import asyncio
from azure.identity.aio import DefaultAzureCredential as AsyncDefaultAzureCredential
from semantic_kernel.agents import AgentGroupChat, AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies import TerminationStrategy
from semantic_kernel.contents import AuthorRole

class FormattingTerminationStrategy(TerminationStrategy):
    async def should_agent_terminate(self, agent, history):
        # Terminate when the formatter agent's message contains the marker 'Formatted Result:'
        return 'Formatted Result:' in history[-1].content

async def format_analysis_result_async(analysis_text):
    # Define agent names and instructions
    ANALYST_NAME = "DataAnalyst"
    ANALYST_INSTRUCTIONS = "You are a data analyst. Your role is to extract key insights from the analysis text. Provide a concise summary of the main findings."
    
    FORMATTER_NAME = "Formatter"
    FORMATTER_INSTRUCTIONS = (
        "You are a formatting expert. Your role is to take the analysis summary and format it in Markdown with clear headers, bullet points, "
        "and a final 'Formatted Result:' section summarizing the insights."
    )
    
    TASK = analysis_text

    ai_agent_settings = AzureAIAgentSettings.create()

    async with (
        AsyncDefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        # Create the data analyst agent
        analyst_definition = await client.agents.create_agent(
            model=ai_agent_settings.model_deployment_name,
            name=ANALYST_NAME,
            instructions=ANALYST_INSTRUCTIONS,
        )
        agent_analyst = AzureAIAgent(client=client, definition=analyst_definition)

        # Create the formatter agent
        formatter_definition = await client.agents.create_agent(
            model=ai_agent_settings.model_deployment_name,
            name=FORMATTER_NAME,
            instructions=FORMATTER_INSTRUCTIONS,
        )
        agent_formatter = AzureAIAgent(client=client, definition=formatter_definition)

        # Create a group chat with a custom termination strategy
        chat = AgentGroupChat(
            agents=[agent_analyst, agent_formatter],
            termination_strategy=FormattingTerminationStrategy(agents=[agent_formatter], maximum_iterations=5),
        )

        try:
            # Add the analysis text as the initial message
            await chat.add_chat_message(messages=TASK)
            print(f"# {AuthorRole.USER}: '{TASK[:60]}...'")
            
            formatted_result = ""
            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")
                if content.role == AuthorRole.ASSISTANT and content.name == FORMATTER_NAME:
                    formatted_result = content.content
            return formatted_result
        finally:
            await chat.reset()
            await client.agents.delete_agent(agent_analyst.id)
            await client.agents.delete_agent(agent_formatter.id)

def format_analysis_result(analysis_text):
    return asyncio.run(format_analysis_result_async(analysis_text))

# To test the formatting conversation, paste your analysis text below and run:
# analysis_text = "Paste your analysis text here..."
# formatted_output = format_analysis_result(analysis_text)
# print("Formatted Output:\n", formatted_output)


c:\Users\armoshar\AppData\Local\anaconda3\envs\gen-ai\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_deployment_name" in AzureAIAgentSettings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


In [3]:
analysis_text = "what is Net Change in Plan Fiduciary Net Position for all yeas?"
formatted_output = format_analysis_result(analysis_text)
print("Formatted Output:\n", formatted_output)

TypeError: AgentChat.add_chat_message() got an unexpected keyword argument 'messages'